<a href="https://colab.research.google.com/github/rvraghvender/DeepLearningProjects/blob/main/ConvolutionNeuralNetworks/NerualArtGeneration/Art_Generation_with_Neural_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning & Art: Neural Style Transfer

## 1. Packages


In [3]:
# Importing relevant packages
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
import matplotlib.pyplot as imshow
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
import pprint
%matplotlib inline

## 2. Problem Statement

Nerual Style Transfer (NST) is one of the most fun and interesting optimzation techniques in deep learing. It merges two images, namely a **"content" image (C)** and **"style" image (S)**, to create a **"generated" image (G) **. The generated image G combines the "content" of the image C with the "style" of image S.

In this assignment, we are going to cobine the Louvre museum in Paris (content image, C) with the impressionist style of Claude Monet (content image S) to generate the following image:

<img src="https://github.com/rvraghvender/DeepLearningProjects/blob/main/ConvolutionNeuralNetworks/NerualArtGeneration/images/louvre_generated.png?raw=true" style="width:750px;height:200px;">

Let's get started!


## Transfer Learning
Neural Style  Transfer (NST) uses a previously tranied convolution network, and builds on top of that. The idea of using a network trained on a different task and applying it to a new task is called transfer learning.

We will be using the epynomously named VGG network from the [original NST paper](https://arxiv.org/abs/1508.06576) published by the Visual Geometry Group at University of Oxford in 2014. Specifically, we'll use VGG-19, a 19-layer version of the VGG network. This model has already been tranied on the very large ImageNet database, and has learned to recoginze a variety of low level features (at the shallow layers) and high level features (at the deeper layer).

Run the below code to load parameters from the VGG model.

In [5]:
tf.random.set_seed(272) # To be consistent with different tries
pp = pprint.PrettyPrinter(indent=4)
img_size = 400
vgg = tf.keras.applications.VGG19(include_top=False,
                                  input_shape=(img_size, img_size, 3),
                                  weights='pretrained-model/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')
vgg.trainable = False
pp.pprint(vgg)

ValueError: ignored